In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.chrome.options import Options 
from selenium.common.exceptions import TimeoutException 
from selenium.webdriver.common.keys import Keys

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import os
from time import sleep

In [2]:
def hj(today = dt.datetime.today()):
    hj = today.strftime('%d-%m-%Y')
    
    return hj

os.mkdir(f'./Azul/{hj()}')
os.mkdir(f'./Latam/{hj()}')
os.mkdir(f'./Gol/{hj()}')

# Azul

In [3]:
service = Service(executable_path = "C:/Users/MATHEUS/Desktop/Biblioteca/Python/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service)

In [4]:
def precificar(p):
    '''
    Transforma uma string de numeros, com decimais separados por virgula, em um float
    '''
    # Remove os pontos de milhares
    temp = p.replace(".", "")
    # Troca as virgulas por pontos e transforma em float
    final = float(temp.replace(",", "."))
    
    return final

def get_dates(today = dt.datetime.today()):
    '''
    Dado o dia de hoje, computa todas as datas de quinze em quinze dias, começando por amanhã
    e terminando em 360 dias. Retorna as datas numa lista em formato %d%m%Y
    '''
    list_days = [1] + [i*15 for i in range(1, 25)]
    dates = [(today + dt.timedelta(days=day)) for day in list_days]
    dates_mdY = [date.strftime('%m/%d/%Y') for date in dates]
    dates_dmY = [date.strftime('%d/%m/%Y') for date in dates]
    dates_hifen = [date.strftime('%d-%m-%Y') for date in dates]
    
    return dates_mdY, dates_dmY, dates_hifen

def get_passagens(partida, destino):
    
    
    dt_mdy, dt_dmy, dt_hifen = get_dates()
    
    for index in range(0, len(dt_mdy)):
        
        print(f"Carregando página do dia {dt_dmy[index]}.")
        driver.get(f'https://www.voeazul.com.br/br/pt/home/selecao-voo?c[0].ds={partida}&c[0].std={dt_mdy[index]}&c[0].as={destino}&p[0].t=ADT&p[0].c=1&p[0].cp=false&f.dl=3&f.dr=3&cc=BRL')
        
        print("Carregando todas as passagens disponíveis...")
        all_loaded = False
        while all_loaded == False:
            try:
                wait = WebDriverWait(driver, 30)
                element = wait.until(EC.element_to_be_clickable((By.ID, "load-more-button")))
                #element = driver.find_element(By.ID, "load-more-button")
                driver.execute_script("arguments[0].click();", element)
            except:
                all_loaded = True
        
        print("Obtendo o código fonte e coletando os dados.")
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        #infos = soup.find_all('div', attrs={'class' : 'info-details css-1aersfg'})
        infos = soup.find_all('p', attrs={'class' : 'flight-leg-info'})
        partidas = soup.find_all('h4', attrs = {'class':'departure css-qtfatn' })
        chegadas = soup.find_all('h4', attrs = {'class':'arrival css-1kh4opz' })
        preços = soup.find_all('div', attrs = {'class':'fare css-t6mqli' })
        
        print("Salvando os dados no DataFrame e salvando em arquivo Pickle.")
        voos = {}
        for i in range(0, len(preços)):

            HORA_PART = partidas[i].text[:5]
            AERO_PART = partidas[i].text[5:]
            HORA_DEST = chegadas[i].text[:5]
            AERO_DEST = chegadas[i].text[5:]

            tag = AERO_PART + '_' + HORA_PART + '-' + AERO_DEST + '_' + HORA_DEST

            passagem = {
                "Companhia": "Azul",
                "Data da Viagem": dt_dmy[index],
                "Horário de Partida" : HORA_PART,
                "Aeroporto de Partida" : AERO_PART,
                "Horário de Destino": HORA_DEST,
                "Aeroporto de Destino": AERO_DEST,
                "Preço (R$)": precificar(preços[i].text[14:]),
                "Detalhes": infos[i].text,
                "Tag": tag
            }

            voos.update({i:passagem})
            
        df = pd.DataFrame.from_dict(voos, orient='index')
            
        df.to_pickle(f"C:/Users/MATHEUS/Desktop/Biblioteca/Python/WebScrapping_Passagens/Azul/{hj()}/{dt_hifen[index]}.pkl")
        
        print(f"Concluído a coleta do dia {dt_dmy[index]} ({index}/{len(dt_mdy) - 1}).\n")

In [5]:
PARTIDA = "REC"
DESTINO = "RIO"

get_passagens(PARTIDA, DESTINO)

driver.quit()

Carregando página do dia 02/08/2023.
Carregando todas as passagens disponíveis...
Obtendo o código fonte e coletando os dados.
Salvando os dados no DataFrame e salvando em arquivo Pickle.
Concluído a coleta do dia 02/08/2023 (0/24).

Carregando página do dia 16/08/2023.
Carregando todas as passagens disponíveis...
Obtendo o código fonte e coletando os dados.
Salvando os dados no DataFrame e salvando em arquivo Pickle.
Concluído a coleta do dia 16/08/2023 (1/24).

Carregando página do dia 31/08/2023.
Carregando todas as passagens disponíveis...
Obtendo o código fonte e coletando os dados.
Salvando os dados no DataFrame e salvando em arquivo Pickle.
Concluído a coleta do dia 31/08/2023 (2/24).

Carregando página do dia 15/09/2023.
Carregando todas as passagens disponíveis...
Obtendo o código fonte e coletando os dados.
Salvando os dados no DataFrame e salvando em arquivo Pickle.
Concluído a coleta do dia 15/09/2023 (3/24).

Carregando página do dia 30/09/2023.
Carregando todas as passage

# Latam

In [6]:
service = Service(executable_path = "C:/Users/MATHEUS/Desktop/Biblioteca/Python/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service)

driver.get("https://www.latamairlines.com/br/pt")

In [7]:
def precificar(p):
    '''
    Transforma uma string de numeros, com decimais separados por virgula, em um float
    '''
    # Remove os pontos de milhares
    temp = p.replace(".", "")
    # Troca as virgulas por pontos e transforma em float
    final = float(temp.replace(",", "."))
    
    return final

def get_dates(today = dt.datetime.today()):
    '''
    Dado o dia de hoje, computa todas as datas de quinze em quinze dias, começando por amanhã
    e terminando em 300 dias. Retorna as datas numa lista em diferentes formatos
    '''
    list_days = [1] + [i*15 for i in range(1, 21)]
    dates = [(today + dt.timedelta(days=day)) for day in list_days]
    dates_Ymd = [date.strftime('%Y-%m-%d') for date in dates]
    dates_dmY = [date.strftime('%d/%m/%Y') for date in dates]
    dates_hifen = [date.strftime('%d-%m-%Y') for date in dates]
    
    return dates_Ymd, dates_dmY, dates_hifen

def get_passagens(partida, destino):
    
    
    dt_Ymd, dt_dmy, dt_hifen = get_dates()
    
    for index in range(0, len(dt_Ymd)):
        
        print(f"Carregando página do dia {dt_dmy[index]}.")
        
        driver.get(f"https://www.latamairlines.com/br/pt/oferta-voos?origin={partida}&inbound=null&outbound={dt_Ymd[index]}T15%3A00%3A00.000Z&destination={destino}&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED")

        driver.implicitly_wait(20)
        
        print("Carregando todas as passagens disponíveis...")
        
        found = False
        while found == False:
            try:
                wait = WebDriverWait(driver, 30)
                temp = wait.until(EC.visibility_of_element_located((By.ID, "WrapperCardFlight0")))
            except:
                print("Erro no carregamento da página. Recarregando...")
                driver.get(f"https://www.latamairlines.com/br/pt/oferta-voos?origin={partida}&inbound=null&outbound={dt_Ymd[index]}T15%3A00%3A00.000Z&destination={destino}&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED")
            else:
                print("Sucesso no carregamento!")
                found = True
                
        print("Obtendo o código fonte e coletando os dados.")
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        preços = soup.find_all('span', attrs = {'class': "display-currencystyle__CurrencyAmount-sc__sc-19mlo29-2 fMjBKP"})
        horarios = soup.find_all('span', attrs = {'class':"card-flightstyle__TextHourFlight-sc__sc-16r5pdw-18 kKmcWo"})
        aeroportos = soup.find_all('span', attrs = {'class': 'card-flightstyle__TextIATA-sc__sc-16r5pdw-19 ksoviQ'})
        detalhes =  soup.find_all("a", attrs = {'class': "sc-hAZoDl fkClzL"})
        
        print("Salvando os dados no DataFrame e salvando em arquivo Pickle.")
        voos = {}

        for i in range(0, len(aeroportos), 2):

            HORA_PART = horarios[i].text
            HORA_DEST = horarios[i + 1].text
            AERO_PART = aeroportos[i].text
            AERO_DEST = aeroportos[i + 1].text

            tag = AERO_PART + '_' + HORA_PART + '-' + AERO_DEST + '_' + HORA_DEST

            passagem = {
                "Companhia": "Latam",
                "Data da Viagem": dt_dmy[index],
                "Horário de Partida" : HORA_PART,
                "Aeroporto de Partida" : AERO_PART,
                "Horário de Destino": HORA_DEST,
                "Aeroporto de Destino": AERO_DEST,
                "Preço (R$)": precificar(preços[i].text),
                "Detalhes": detalhes[i//2].text,
                "Tag": tag
                }

            voos.update({(i//2):passagem})
            
        df = pd.DataFrame.from_dict(voos, orient='index')
            
        df.to_pickle(f"C:/Users/MATHEUS/Desktop/Biblioteca/Python/WebScrapping_Passagens/Latam/{hj()}/{dt_hifen[index]}.pkl")
        
        print(f"Concluído a coleta do dia {dt_dmy[index]} ({index}/{len(dt_Ymd) - 1}).\n")

In [8]:
PARTIDA = "REC"
DESTINO = "RIO"

get_passagens(PARTIDA, DESTINO)

driver.quit()

Carregando página do dia 02/08/2023.
Carregando todas as passagens disponíveis...
Sucesso no carregamento!
Obtendo o código fonte e coletando os dados.
Salvando os dados no DataFrame e salvando em arquivo Pickle.
Concluído a coleta do dia 02/08/2023 (0/20).

Carregando página do dia 16/08/2023.
Carregando todas as passagens disponíveis...
Sucesso no carregamento!
Obtendo o código fonte e coletando os dados.
Salvando os dados no DataFrame e salvando em arquivo Pickle.
Concluído a coleta do dia 16/08/2023 (1/20).

Carregando página do dia 31/08/2023.
Carregando todas as passagens disponíveis...
Sucesso no carregamento!
Obtendo o código fonte e coletando os dados.
Salvando os dados no DataFrame e salvando em arquivo Pickle.
Concluído a coleta do dia 31/08/2023 (2/20).

Carregando página do dia 15/09/2023.
Carregando todas as passagens disponíveis...
Sucesso no carregamento!
Obtendo o código fonte e coletando os dados.
Salvando os dados no DataFrame e salvando em arquivo Pickle.
Concluído 

# Gol

In [9]:
service = Service(executable_path = "C:/Users/MATHEUS/Desktop/Biblioteca/Python/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service)

driver.get("https://b2c.voegol.com.br/compra/")

In [10]:
def precificar(str0):
    str1 = str0.replace(u'R$\xa0', u'')
    str2 = str1.replace(' ', '')
    str3 = str2.replace('.', '')
    final = str3.replace(',', '.')
    
    return float(final)

def get_dates(today = dt.datetime.today()):
    '''
    Dado o dia de hoje, computa todas as datas de quinze em quinze dias, começando por amanhã
    e terminando em 360 dias. Retorna as datas numa lista em formato %d%m%Y
    '''
    list_days = [1] + [i*15 for i in range(1, 21)]
    dates = [(today + dt.timedelta(days=day)) for day in list_days]
    
    #dates_search = {months[date.month] + " " + str(date.year):date.strftime('%d%m%Y') for date in dates}
    dates_search = [date.strftime('%d%m%Y') for date in dates]
    dates_dmY = [date.strftime('%d/%m/%Y') for date in dates]
    dates_hifen = [date.strftime('%d-%m-%Y') for date in dates]
    
    return dates_search, dates_dmY, dates_hifen

In [11]:
dates_search, dt_dmY, dates_hifen = get_dates()

for DATA in dates_search:
    driver.get("https://b2c.voegol.com.br/compra/")
    index = dates_search.index(DATA)
    if index == 0:
        print("Selecionando local de partida e destino.")
        # Aperta o Botão de Selecionar Trecho
        driver.find_element(By.XPATH, '//*[@id="compra"]/b2c-shopping/section/form/div[1]/div/div[1]/div[1]/b2c-dropdown/div/div[1]/fieldset').click()
        wait = WebDriverWait(driver, 5)
        # Seleciona Ida
        driver.find_element(By.XPATH, '//*[@id="dropdown-trecho"]/ul/li[2]/b2c-radio/div/label/span').click()

        # Aperta o botão de Saindo de
        driver.find_element(By.XPATH, '//*[@id="input-saindo-de"]').click()
        # Aperta o botão de todos os aeroportos
        driver.find_element(By.XPATH, '//*[@id="dropdown-saindo-de"]/b2c-list-cta/div/div/button').click()
        # Aperta o botão de aeroporto doméstico
        driver.find_element(By.XPATH, '//*[@id="modal-departure"]/div[2]/div[2]/b2c-tabs-list/div/ul/li[1]/a').click()
        # Seleciona o aeroporto de Recife
        driver.find_element(By.XPATH, '//*[@id="modal-departure"]/div[2]/div[2]/b2c-tabs-list/div/div[1]/ul/li[87]/a').click()

        # Aperta o botão de Indo para
        driver.find_element(By.XPATH, '//*[@id="input-indo-para"]').click()
        # Aperta o botão de todos os aeroportos
        driver.find_element(By.XPATH, '//*[@id="dropdown-indo-para"]/b2c-list-cta/div/div[1]/button').click()
        # Aperta o botão de aeroporto doméstico
        driver.find_element(By.XPATH, '//*[@id="modal-arrival"]/div[2]/div[2]/b2c-tabs-list/div/ul/li[1]/a').click()
        # Seleciona todos os aeroportos do Rio de Janeiro
        driver.find_element(By.XPATH, '//*[@id="modal-arrival"]/div[2]/div[2]/b2c-tabs-list/div/div[1]/ul/li[93]/a').click()
    
       
    #text = MES
    #//*[@id="datePicker-departureDate"]/b2c-calendar/div/div[1]/div/div[1]/div/p/span
    #driver.find_element(By.XPATH, '//*[@id="departureDate"]').click()
    #curr_month = driver.find_element(By.XPATH , '//*[@id="datePicker-departureDate"]/b2c-calendar/div/div[1]/div/div[1]/div/p').text
    
    #while text != curr_month:
    #    next_btn = driver.find_element(By.XPATH, '//*[@id="datePicker-departureDate"]/b2c-calendar/div/div[1]/div/div[1]/div/button[2]')
    #    driver.execute_script("arguments[0].click();", next_btn)
    #    curr_month = driver.find_element(By.XPATH , '//*[@id="datePicker-departureDate"]/b2c-calendar/div/div[1]/div/div[1]/div/p/span').text
    
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    #driver.find_element(By.ID , 'departureDate' + DATA).click()
    print("Selecionando a data de viagem")
    driver.find_element(By.XPATH, '//*[@id="departureDate"]').click()
    date_found = False
    while date_found == False:
        try:
            element = driver.find_element(By.ID, 'departureDate' + DATA)
            driver.execute_script("arguments[0].click();", element)
        except:
            next_btn = driver.find_element(By.XPATH, '//*[@id="datePicker-departureDate"]/b2c-calendar/div/div[1]/div/div[1]/div/button[2]')
            driver.execute_script("arguments[0].click();", next_btn)
            #curr_month = driver.find_element(By.XPATH , '//*[@id="datePicker-departureDate"]/b2c-calendar/div/div[1]/div/div[1]/div/p/span').text
        else:
            date_found = True
    
    driver.find_element(By.XPATH,'/html/body/app-root/b2c-motor/main/b2c-tabs-router/div[2]/b2c-shopping/section/form/div[2]/div[2]/div[2]/b2c-button').click()
    
    print("Esperando todas as passagens carregarem...")
    sleep(60)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    print("Salvando todas as passagens em arquivo pickle.")
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    preços = soup.find_all('span', attrs = {'class': 'a-desc__value a-desc__value--price'})
    part_dest = soup.find_all('span', attrs = {'class': 'a-desc__value'})
    info = soup.find_all('a', attrs = {'class': 'a-desc__value m-bar-product__link ng-star-inserted'})
    
    voos = {}
    
    # As vezes as promoções aparecem no elemento part_dest, aumentando o numero de saltos que devemos
    # fazer para salvar os dados de 4 para 5. O código abaixo chega se a variável tem tamanho divisivel
    # por 4 ou 5
    
    elements = 4    
    if len(part_dest)%4 == 0:
        elements = 4
    if len(part_dest)%5 == 0:
        elements = 5
    
    for i in range(0, len(part_dest), elements):
        HORA_PART = part_dest[i].text[6:]
        HORA_DEST = part_dest[i+1].text[6:]
        AERO_PART = part_dest[i].text[0:3]
        AERO_DEST = part_dest[i+1].text[0:3]

        tag = AERO_PART + '_' + HORA_PART + '-' + AERO_DEST + '_' + HORA_DEST

        passagem = {
                    "Companhia": "Gol",
                    "Data da Viagem": dt_dmY[index],
                    "Horário de Partida" : HORA_PART,
                    "Aeroporto de Partida" : AERO_PART,
                    "Horário de Destino": HORA_DEST,
                    "Aeroporto de Destino": AERO_DEST,
                    "Preço (R$)": precificar(preços[i//elements].text),
                    "Detalhes": info[i//elements].text,
                    "Tag": tag
                    }
        voos.update({(i//elements):passagem})
    df = pd.DataFrame.from_dict(voos, orient='index')
        
    df.to_pickle(f"C:/Users/MATHEUS/Desktop/Biblioteca/Python/WebScrapping_Passagens/Gol/{hj()}/{dates_hifen[index]}.pkl")
        
    print(f"Concluído a coleta do dia {dt_dmY[index]} ({index + 1}/{len(dt_dmY)}).\n")
    
driver.quit()

Selecionando local de partida e destino.
Selecionando a data de viagem
Esperando todas as passagens carregarem...
Salvando todas as passagens em arquivo pickle.
Concluído a coleta do dia 02/08/2023 (1/21).

Selecionando a data de viagem
Esperando todas as passagens carregarem...
Salvando todas as passagens em arquivo pickle.
Concluído a coleta do dia 16/08/2023 (2/21).

Selecionando a data de viagem
Esperando todas as passagens carregarem...
Salvando todas as passagens em arquivo pickle.
Concluído a coleta do dia 31/08/2023 (3/21).

Selecionando a data de viagem
Esperando todas as passagens carregarem...
Salvando todas as passagens em arquivo pickle.
Concluído a coleta do dia 15/09/2023 (4/21).

Selecionando a data de viagem
Esperando todas as passagens carregarem...
Salvando todas as passagens em arquivo pickle.
Concluído a coleta do dia 30/09/2023 (5/21).

Selecionando a data de viagem
Esperando todas as passagens carregarem...
Salvando todas as passagens em arquivo pickle.
Concluído